# 1. Setup Planning Analytics Credentials and Watson Machine Learning Credentials

In [1]:
tm1_credentials = {
    "address":"", 
    "port": "",
    "user": "", 
    "password": "", 
    "ssl": ""
}

In [2]:
wml_credentials = {
    "apikey":"",
    "instance_id":"",
    "url": ""
}

# 2. Extract Scoring View from Planning Analytics

In [4]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import build_pandas_dataframe_from_cellset

In [5]:
# Import data from PA 
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user'], password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    data = tm1.cubes.cells.execute_view(cube_name='OPS Sales', view_name='Predict View', private = False)
    df = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [7]:
df.head()

,Time Date,Version,Account,Product,Store,OPS Sales Source,OPS Sales Measure,Values
540,01012019,Actual,4999,2477202,NSW_CW_ST0125,Base,Amount,None
90,01012019,Actual,4999,2477202,QLD_CW_ST0005,Base,Amount,None
990,01012019,Actual,4999,2477202,VIC_CW_ST0411,Base,Amount,None
450,01012019,Actual,4999,2511675,NSW_CW_ST0125,Base,Amount,None
0,01012019,Actual,4999,2511675,QLD_CW_ST0005,Base,Amount,None


# 3. Data Preprocessing

In [8]:
import pandas as pd

In [12]:
# Date Preprocessing - reformat date string
df['Date'] = df['Time Date'].apply(lambda x: x[4:]+x[2:4]+x[:2])

# Convert to Datetime
df['Date'] = pd.to_datetime(df['Date'])

In [13]:
df['Day'] = df['Date'].apply(lambda x: x.day)
df['Day of Week'] = df['Date'].apply(lambda x: x.dayofweek)
df['Day of Year'] = df['Date'].apply(lambda x: x.dayofyear)
df['Month'] = df['Date'].apply(lambda x: x.month)
df['Quarter End'] = df['Date'].apply(lambda x: x.is_quarter_end)
df['Month End'] = df['Date'].apply(lambda x: x.is_month_end)
df['Year'] = df['Date'].apply(lambda x: x.year)

In [14]:
df.head()

,Time Date,Version,Account,Product,Store,OPS Sales Source,OPS Sales Measure,Values,Date,Day,Day of Week,Day of Year,Month,Quarter End,Month End,Year
540,01012019,Actual,4999,2477202,NSW_CW_ST0125,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019
90,01012019,Actual,4999,2477202,QLD_CW_ST0005,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019
990,01012019,Actual,4999,2477202,VIC_CW_ST0411,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019
450,01012019,Actual,4999,2511675,NSW_CW_ST0125,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019
0,01012019,Actual,4999,2511675,QLD_CW_ST0005,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019


In [15]:
# Update product column
df['Product'] = 'PROD'+df['Product']

In [19]:
abt = df.drop(['Time Date', 'Date', 'Version', 'Account', 'OPS Sales Source', 'OPS Sales Measure', 'Values'], axis=1)

In [20]:
abt.head()

,Product,Store,Day,Day of Week,Day of Year,Month,Quarter End,Month End,Year
540,PROD2477202,NSW_CW_ST0125,1,1,1,1,False,False,2019
90,PROD2477202,QLD_CW_ST0005,1,1,1,1,False,False,2019
990,PROD2477202,VIC_CW_ST0411,1,1,1,1,False,False,2019
450,PROD2511675,NSW_CW_ST0125,1,1,1,1,False,False,2019
0,PROD2511675,QLD_CW_ST0005,1,1,1,1,False,False,2019


In [21]:
scoring = abt.values.tolist()

In [22]:
scoring

[['PROD2477202', 'NSW_CW_ST0125', 1, 1, 1, 1, False, False, 2019],
 ['PROD2477202', 'QLD_CW_ST0005', 1, 1, 1, 1, False, False, 2019],
 ['PROD2477202', 'VIC_CW_ST0411', 1, 1, 1, 1, False, False, 2019],
 ['PROD2511675', 'NSW_CW_ST0125', 1, 1, 1, 1, False, False, 2019],
 ['PROD2511675', 'QLD_CW_ST0005', 1, 1, 1, 1, False, False, 2019],
 ['PROD2511675', 'VIC_CW_ST0411', 1, 1, 1, 1, False, False, 2019],
 ['PROD2662978', 'NSW_CW_ST0125', 1, 1, 1, 1, False, False, 2019],
 ['PROD2662978', 'QLD_CW_ST0005', 1, 1, 1, 1, False, False, 2019],
 ['PROD2662978', 'VIC_CW_ST0411', 1, 1, 1, 1, False, False, 2019],
 ['PROD2673107', 'NSW_CW_ST0125', 1, 1, 1, 1, False, False, 2019],
 ['PROD2673107', 'QLD_CW_ST0005', 1, 1, 1, 1, False, False, 2019],
 ['PROD2673107', 'VIC_CW_ST0411', 1, 1, 1, 1, False, False, 2019],
 ['PROD2679547', 'NSW_CW_ST0125', 1, 1, 1, 1, False, False, 2019],
 ['PROD2679547', 'QLD_CW_ST0005', 1, 1, 1, 1, False, False, 2019],
 ['PROD2679547', 'VIC_CW_ST0411', 1, 1, 1, 1, False, False, 20

# 4. Score Data

In [23]:
!pip install watson-machine-learning-client

In [25]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [26]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [27]:
iam_token = client.wml_token

In [28]:
ml_instance_id = wml_credentials['instance_id']

In [29]:
import urllib3, requests, json

# NOTE: generate iam_token and retrieve ml_instance_id based on provided documentation	
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token, 'ML-Instance-ID': ml_instance_id}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": ["Product", "Store", "Day", "Day of Week", "Day of Year", "Month", "Quarter End", "Month End", "Year"], "values": scoring}]}

response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/v4/deployments/74c05368-26b2-4260-accc-eb973efaf40d/predictions', json=payload_scoring, headers=header)

In [32]:
predictions = response_scoring.json()

In [33]:
df['Predictions'] = [x[0] for x in predictions['predictions'][0]['values']]

In [34]:
df.head()

,Time Date,Version,Account,Product,Store,OPS Sales Source,OPS Sales Measure,Values,Date,Day,Day of Week,Day of Year,Month,Quarter End,Month End,Year,Predictions
540,01012019,Actual,4999,PROD2477202,NSW_CW_ST0125,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019,1619.833594
90,01012019,Actual,4999,PROD2477202,QLD_CW_ST0005,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019,1860.443188
990,01012019,Actual,4999,PROD2477202,VIC_CW_ST0411,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019,2080.000000
450,01012019,Actual,4999,PROD2511675,NSW_CW_ST0125,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019,1892.333728
0,01012019,Actual,4999,PROD2511675,QLD_CW_ST0005,Base,Amount,None,2019-01-01,1,1,1,1,False,False,2019,2167.481152


# 5. Push Data Back to PA

In [35]:
cellset = {(x[0], 'AutoAI Forecast', x[2], x[3][4:], x[4], 'Base', 'Amount'):x[16] for x in df.values.tolist()}

In [36]:
cellset

{('01012019',
  'AutoAI Forecast',
  '4999',
  '2477202',
  'NSW_CW_ST0125',
  'Base',
  'Amount'): 1619.83359375,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2477202',
  'QLD_CW_ST0005',
  'Base',
  'Amount'): 1860.4431884765625,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2477202',
  'VIC_CW_ST0411',
  'Base',
  'Amount'): 2080.0,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2511675',
  'NSW_CW_ST0125',
  'Base',
  'Amount'): 1892.3337280273438,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2511675',
  'QLD_CW_ST0005',
  'Base',
  'Amount'): 2167.48115234375,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2511675',
  'VIC_CW_ST0411',
  'Base',
  'Amount'): 2444.0,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2662978',
  'NSW_CW_ST0125',
  'Base',
  'Amount'): 498.6855529785156,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2662978',
  'QLD_CW_ST0005',
  'Base',
  'Amount'): 568.0833984375,
 ('01012019',
  'AutoAI Forecast',
  '4999',
  '2662978',
  'VIC_CW_ST0411',

In [38]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user'], password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    tm1.cubes.cells.write_values('OPS Sales', cellset, dimensions=['Time Date', 'Version', 'Account', 'Product', 'Store', 'OPS Sales Source', 'OPS Sales Measure'])